In [1]:
import numpy as np
import pandas as pd
import sympy as sp
from fractions import Fraction

In [2]:
NUM_PLAYERS = 3
PLAYERS = list(range(1, NUM_PLAYERS+1))
# all possible (giver, receiver) pairs
OUTCOME_PROB = Fraction(1, NUM_PLAYERS * (NUM_PLAYERS-1))  # 1/6
CANDIDATE_MOVES = [(g, r) for idx, g in enumerate(PLAYERS) for r in PLAYERS if g != r]
LOSER_IDX = 0

In [3]:
def applyMove(initState, move):
    g, r = move
    initWealth = list(initState)
    transferAmt = min(initWealth[g-1], initWealth[r-1])  # 0-indexed
    initWealth[g-1] -= transferAmt  # give
    initWealth[r-1] += transferAmt  # receive
    return tuple(initWealth)


def getChildStates(initState):
    """
    Applies all possible moves to the initial state.
    Returns the child states.
    """
    return [applyMove(initState, move) for move in CANDIDATE_MOVES]

def getCoef(initState):
    """
    Returns the coefficients for a first-step analysis equation,
    where the hitting probability of the initial state is the subject.
    """
    # assign all states the individual outcome probability
    childStates = getChildStates(initState)
    stateCoefs = {s:OUTCOME_PROB for s in childStates}
    const = 0
    
    for state in stateCoefs.copy():
        # prune the 0 hitting prob states
        if zeroHittingProb(state):
            stateCoefs.pop(state)
        # convert the hitting prob 1 states to constants
        elif alreadyHit(state):
            const += stateCoefs[state]
            stateCoefs.pop(state)
        # optimsation 1: convert equal wealth states to constants
        elif equalWealth(state):
            const += stateCoefs[state] / NUM_PLAYERS
            stateCoefs.pop(state)
    
    # optimization 2: combine symmetrical states
    copy = list(stateCoefs.copy().keys())
    n = len(copy)
    for i in range(n):
        for j in range(i+1, n):
            s1, s2 = copy[i], copy[j]
            if (symmetricalStates(s1, s2)):
                combinedProb = stateCoefs.pop(s1) + stateCoefs.pop(s2)
                stateCoefs[standardiseState(s1)] = combinedProb
    
    return standardiseStates(stateCoefs), const

def zeroHittingProb(state):
    """
    Returns true if the given state cannot reach the state of Player 1 being the loser, false otherwise.
    
    """
    return (0 in state) and (state[LOSER_IDX] > 0)

def alreadyHit(state):
    """
    Returns true if the given state reaches the state of Player 1 being the loser wp1, false otherwise.
    
    """
    return state[LOSER_IDX] == 0

def symmetricalStates(s1, s2):
    return s1[0] == s2[0] and set(s1[1:]) == set(s2[1:])

def standardiseState(s):
    # convention: keep the smaller value at the front
    return s if s[1] <= s[2] else (s[0], s[2], s[1])

def standardiseStates(stateCoefs):
    return {standardiseState(s):p for s, p in stateCoefs.items()}
    
def equalWealth(s):
    return s[0] == s[1] and s[1] == s[2]

In [55]:
from sympy import *
from sympy import nsolve
from sympy.solvers import solve
from ast import literal_eval as make_tuple  # for parsing state as a tuple

In [70]:
def getEquation(initState, stateCoefs, const):
    # equation will be in the form of human readable equalities (like how we normally write down)
    symbols = [stateToSymbol(s) for s in stateCoefs]
    eqn = Eq(stateToSymbol(initState), getExpr(stateCoefs, const))
    return eqn, symbols
    
def stateToSymbol(state):
    return Symbol(f'h{state}')

def symbolToState(symbol):
    return make_tuple(str(symbol)[1:])

def getExpr(stateCoefs, const):
    expr = const
    for s in stateCoefs:
        expr = Add(expr, stateCoefs[s] * stateToSymbol(s))
    return expr

def formatEquation(eqn):
    return f'{eqn.lhs} = {eqn.rhs}'

def formatSolutions(sol):
    return [(symbolToState(s), p) for s, p in sol.items()]

In [63]:
class LoserAnalysis():
    def __init__(self, initState):
        self.initState = standardiseState(initState)
        self.generatedStates = set([initState])
        self.equations = []
        self.symbols = set() # or can encode all generated states at once in the end
    
    def getEquations(self):
        self.recurseGenerateEqns(self.initState)
        return self.equations
        
    def recurseGenerateEqns(self, initState):
        childStatesProb, const = getCoef(initState)  # state:prob pairs, under unified representation
        newStates = set(childStatesProb).difference(self.generatedStates)
        
        # compute for the current state
        # do this before the recursion to close off the loop
        eqn, symbols = getEquation(initState, childStatesProb, const)
        self.equations.append(eqn)

        # base case: no new states
        if not newStates:
            return

        self.symbols = self.symbols.union(symbols)
        self.generatedStates = self.generatedStates.union(newStates)
        
        # recurse
        for state in newStates:
            self.recurseGenerateEqns(state)
    
    def solveEqns(self):
        self.getEquations()
        return solve(self.equations, self.symbols)
    
    # numerical approx
#     def nsolveEqns(self):
#         self.getEquations()
#         exprs = 
#         return nsolve(self.equations, self.symbols, np.repeat(1, len(self.symbols)))
    
    def exportEqns(self, filename):
        """
        Exports the first step analysis equations to a text file, 
        using the provided filename.
        """
        self.getEquations()
        fp = open(filename, 'w')
        for e in self.equations:
            fp.write(formatEquation(e) + '\n')
        fp.close()

In [64]:
LoserAnalysis((1,2,3)).solveEqns().items()

dict_items([(h(1, 1, 4), 44/93), (h(1, 2, 3), 113/217), (h(2, 1, 3), 69/217), (h(3, 1, 2), 5/31), (h(4, 1, 1), 5/93)])

In [65]:
LoserAnalysis((4,2,3)).solveEqns()

{h(1, 2, 6): 8298/15019,
 h(1, 4, 4): 7379/15019,
 h(2, 1, 6): 5644/15019,
 h(2, 2, 5): 6727/15019,
 h(2, 3, 4): 7118/15019,
 h(3, 2, 4): 4695/15019,
 h(4, 1, 4): 3820/15019,
 h(4, 2, 3): 3206/15019,
 h(5, 2, 2): 1565/15019,
 h(6, 1, 2): 1077/15019}

In [76]:
LoserAnalysis((2,2,2)).solveEqns()

[]

In [73]:
MAX_AMT = 10
MIN_AMT = 1
FILENAME = 'player1LoseProbs.csv'

In [74]:
import csv

In [75]:
# aggregate to text file
fp = open(FILENAME, 'w')
writer = csv.writer(fp)

stateProbs = dict()  # states as symbols : hitting prob

for i in range(MIN_AMT, MAX_AMT+1):
    for j in range(2, MAX_AMT+1):
        for k in range(3, MAX_AMT+1):
            initState = (i, j, k)
            sols = LoserAnalysis(initState).solveEqns()
            if sols:
                newStates = set(sols).difference(stateProbs)
                for s in newStates:
                    stateProbs[s] = sols[s]
                
# export to CSV
writer.writerows(formatSolutions(stateProbs))

fp.close()

KeyboardInterrupt: 